In [1]:
from keras.layers import Input, Conv2D, AveragePooling2D, MaxPooling2D, Dense, Flatten
from keras import layers, backend, utils, optimizers, datasets
from keras.models import Model, load_model
import tensorflow as tf

from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import h5py

Using TensorFlow backend.


In [2]:
train_input_path = 'MNIST-28x28-train.txt'
test_input_path = 'MNIST-28x28-test.txt'
preprocessed_input_path = 'Preprocessed-MNIST-train_test.npz'
model_dir = 'ReLU/' 
weight_path = ''
train_dataset_size = 60000
test_dataset_size = 10000
image_size = 784  # 28 x 28 pixels

In [3]:
# MNIST
def MNIST_reader(input_name, data_size_num, image_size_num):
    content = []
    with open(input_name, 'r') as file:
        for line in file:
            content.append(line)

    input_pixels = np.zeros([data_size_num, image_size_num])
    labels = np.zeros([data_size_num])

    for image in range(0, len(content)):
        coordinates = np.asarray(content[image].split('\t'))
        for coordinate in coordinates[2:]:
            if coordinate == '':
                break
            pair = coordinate.split(':')
            input_pixels[image, int(pair[0])] = pair[1]
            labels[image] = coordinates[0]
    
    return input_pixels, labels

In [4]:
# Upper and lower padding
def ULpadding(data_list):
    padded = []
    for dataset in data_list:
        reshaped = dataset.reshape(dataset.shape[0], 28, 28, 1)
        padded_reshaped = np.insert(reshaped, 0, 0, axis=1)
        padded.append(np.insert(padded_reshaped, 0, 0, axis=2)/255)
    return (*padded,)

# Converts integer labels to one hot encoding
def one_hot(label_list):
    encoded = []
    for label in label_list:
        encoded.append(utils.to_categorical(label))
    return (*encoded,)

# CryptoNet's Testing Network
def Simplified_CryptoNet(input_size, custom):
    input_shape = Input(shape=(input_size))
    if custom:
        conv1 = Conv2D(filters=5, kernel_size=(5,5), strides=(2,2), activation=approxReLU)(input_shape)
    else:
        conv1 = Conv2D(filters=5, kernel_size=(5,5), strides=(2,2), activation='relu')(input_shape)
    
    flat = Flatten()(conv1) 
    
    if custom:
        dense = Dense(100, activation=approxReLU)(flat)
    else:
        dense = Dense(100, activation='relu')(flat)
        
    output_shape = Dense(10, activation='softmax')(dense)
    
    model = Model(inputs=input_shape, outputs=output_shape)
    model.summary()
    return model

def approxReLU(x):
    return 0.125*x**2 + 0.25*x + 0.5
    #return 0.1997*x**2 + 0.5002*x + 0.1992

def squared(x):
    return x**2

def train(model, train_images, train_labels, epoch=10, val_split=0.2, **kwargs):
    sgd = optimizers.SGD(lr=0.0095)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['acc'])
    model.fit(x=train_images, y=train_labels, epochs=epoch, verbose=1, validation_split=val_split)

def avg_ensemble(predicted1, predicted2, predicted3):
    return (predicted1 + predicted2 + predicted3)/3

def max_ensemble(predicted1, predicted2, predicted3):
    maximum = np.maximum(predicted1, predicted2)
    return np.maximum(maximum, predicted3)

# Splits same training for testing
def with_train_data(data_list, label_list):
    train_x1, test_x1, train_y1, test_y1 = train_test_split(datalist[1], label_list[1], test_size=0.2)
    train_x2, test_x2, train_y2, test_y2 = train_test_split(datalist[2], label_list[2], test_size=0.2)
    train_x3, test_x3, train_y3, test_y3 = train_test_split(datalist[3], label_list[3], test_size=0.2)

    test_images = np.concatenate(test_x1, test_x2, test_x3, axis=0)
    test_labels = np.concatenate(test_y1, test_y2, test_y3, axis=0)
    
    train_list = [data_list[0], train_x1, train_x2, train_x3, test_images]
    labels_list = [label_list[0], train_y1, train_y2, train_y3, test_labels]
    
    return with_train_and_test_data(train_list, labels_list)

# Uses different sets for training and testing
def with_train_and_test_data(data_list, label_list, epoch):
    whole_train, train_x1, train_x2, train_x3, test_images = ULpadding(data_list)
    whole_label, train_y1, train_y2, train_y3, test_labels = one_hot(label_list)
    
    train_list = [whole_train, train_x1, train_x2, train_x3, test_images]
    labels_list = [whole_label, train_y1, train_y2, train_y3, test_labels]
    epoch_list = epoch_ratios(epoch, train_list) 
    
    return epoch_list, train_list, labels_list
    
# Creates ratios for epoch such that smaller datasets are traversed 
# more than larger datasets to compensate
def epoch_ratios(epoch, train_shapes):
    ratio_list = []
    
    whole_shape = train_shapes[0].shape[0]
    
    ratio1 = whole_shape/train_shapes[1].shape[0]
    ratio2 = whole_shape/train_shapes[2].shape[0]
    ratio3 = whole_shape/train_shapes[3].shape[0]
    
    epoch_whole = epoch
    
    ratio_list.append(epoch_whole) # whole dataset
    ratio_list.append(int(ratio1 * epoch_whole)) # model 1
    ratio_list.append(int(ratio2 * epoch_whole)) # model 2
    ratio_list.append(int(ratio3 * epoch_whole)) # model 3
    
    return ratio_list   

def train_dataset(epoch_list, img_list, label_list, custom=False):
    # Whole Dataset
    model = Simplified_CryptoNet(img_list[0].shape[1:], custom)
    train(model, img_list[0], label_list[0], epoch=epoch_list[0])

    # Ensemble
    model1 = Simplified_CryptoNet(img_list[1].shape[1:], custom)
    model2 = Simplified_CryptoNet(img_list[2].shape[1:], custom)
    model3 = Simplified_CryptoNet(img_list[3].shape[1:], custom)

    train(model1, img_list[1], label_list[1], epoch=epoch_list[1])
    train(model2, img_list[2], label_list[2], epoch=epoch_list[2])
    train(model3, img_list[3], label_list[3], epoch=epoch_list[3])
    
    return model, model1, model2, model3

def accuracy_test(model_list, test_img, test_label):
    whole_acc = 0
    avg_ensemble_acc = 0
    max_ensemble_acc = 0
    model1_acc = 0
    model2_acc = 0
    model3_acc = 0

    whole = model_list[0].predict(test_img)
    predict1 = model_list[1].predict(test_img)
    predict2 = model_list[2].predict(test_img)
    predict3 = model_list[3].predict(test_img)

    avg = avg_ensemble(predict1, predict2, predict3)
    maximum = max_ensemble(predict1, predict2, predict3)  
    
    true_label = np.argmax(test_label, axis=1)
    whole_label = np.argmax(whole, axis=1)
    avg_label = np.argmax(avg, axis=1)
    max_label = np.argmax(maximum, axis=1)
    model1 = np.argmax(predict1, axis=1)
    model2 = np.argmax(predict2, axis=1)
    model3 = np.argmax(predict3, axis=1)
    
    for index in range(0, test_img.shape[0]):
        if whole_label[index] == true_label[index]:
            whole_acc += 1
        if avg_label[index] == true_label[index]:
            avg_ensemble_acc += 1
        if max_label[index] == true_label[index]:
            max_ensemble_acc += 1
        if model1[index] == true_label[index]:
            model1_acc += 1
        if model2[index] == true_label[index]:
            model2_acc += 1
        if model3[index] == true_label[index]:
            model3_acc += 1
    
    return test_img.shape[0], whole_acc, avg_ensemble_acc, max_ensemble_acc, model1_acc, model2_acc, model3_acc

In [ ]:
# Read in input
train_input_pixels, train_labels = MNIST_reader(train_input_path, train_dataset_size, image_size)
test_input_pixels, test_labels = MNIST_reader(test_input_path, test_dataset_size, image_size)

np.savez(preprocessed_input_path, train_img=train_input_pixels, train_lab=train_labels,
        test_img=test_input_pixels, test_lab=test_labels)

In [5]:
# Load in saved preprocessed np arrays
with np.load(preprocessed_input_path, 'r') as preprocessed:
    train_input_pixels = preprocessed['train_img']
    train_labels = preprocessed['train_lab']
    test_input_pixels = preprocessed['test_img']
    test_labels = preprocessed['test_lab']

In [6]:
# MNIST Training Splicing
zero_images = []
zero_labels = []
one_images = []
one_labels = []
two_images = []
two_labels = []
three_images = []
three_labels = []
four_images = []
four_labels = []
five_images = []
five_labels = []
six_images = []
six_labels = []
seven_images = []
seven_labels = []
eight_images = []
eight_labels = []
nine_images = []
nine_labels = []

zero = []
one = []
two = []
three = []
four = []
five = []
six = []
seven = []
eight = []
nine = []

for idx in range(train_dataset_size):
    if train_labels[idx] == 0:
        zero_images.append(train_input_pixels[idx])
        zero_labels.append(train_labels[idx])
        zero.append((train_input_pixels[idx],train_labels[idx]))
    elif train_labels[idx] == 1:
        one_images.append(train_input_pixels[idx])
        one_labels.append(train_labels[idx])
        one.append((train_input_pixels[idx],train_labels[idx]))
    elif train_labels[idx] == 2:
        two_images.append(train_input_pixels[idx])
        two_labels.append(train_labels[idx])
        two.append((train_input_pixels[idx],train_labels[idx]))
    elif train_labels[idx] == 3:
        three_images.append(train_input_pixels[idx])
        three_labels.append(train_labels[idx])
        three.append((train_input_pixels[idx],train_labels[idx]))
    elif train_labels[idx] == 4:
        four_images.append(train_input_pixels[idx])
        four_labels.append(train_labels[idx])
        four.append((train_input_pixels[idx],train_labels[idx]))
    elif train_labels[idx] == 5:
        five_images.append(train_input_pixels[idx])
        five_labels.append(train_labels[idx])
        five.append((train_input_pixels[idx],train_labels[idx]))
    elif train_labels[idx] == 6:
        six_images.append(train_input_pixels[idx])
        six_labels.append(train_labels[idx])
        six.append((train_input_pixels[idx],train_labels[idx]))
    elif train_labels[idx] == 7:
        seven_images.append(train_input_pixels[idx])
        seven_labels.append(train_labels[idx])
        seven.append((train_input_pixels[idx],train_labels[idx]))
    elif train_labels[idx] == 8:
        eight_images.append(train_input_pixels[idx])
        eight_labels.append(train_labels[idx])
        eight.append((train_input_pixels[idx],train_labels[idx]))
    else:
        nine_images.append(train_input_pixels[idx])
        nine_labels.append(train_labels[idx])
        nine.append((train_input_pixels[idx],train_labels[idx]))

num = [len(zero), len(one), len(two), len(three), len(four), len(five), len(six), len(seven), len(eight), len(nine)]
print(num)

dataset1_images = []
dataset1_labels = []
dataset2_images = []
dataset2_labels = []
dataset3_images = []
dataset3_labels = []

dataset1_images += zero_images[:5000]
dataset1_images += one_images[:5000]
dataset1_images += two_images[:5000]
dataset1_images += three_images[:250]
dataset1_images += four_images[:250]
dataset1_images += five_images[:150]
dataset1_images += six_images[:250]
dataset1_images += seven_images[:250]
dataset1_images += eight_images[:250]
dataset1_images += nine_images[:250]

dataset1_labels += zero_labels[:5000]
dataset1_labels += one_labels[:5000]
dataset1_labels += two_labels[:5000]
dataset1_labels += three_labels[:250]
dataset1_labels += four_labels[:250]
dataset1_labels += five_labels[:150]
dataset1_labels += six_labels[:250]
dataset1_labels += seven_labels[:250]
dataset1_labels += eight_labels[:250]
dataset1_labels += nine_labels[:250]

dataset2_images += zero_images[5000:5250]
dataset2_images += one_images[5000:5250]
dataset2_images += two_images[5000:5250]
dataset2_images += three_images[250:5250]
dataset2_images += four_images[250:5250]
dataset2_images += five_images[150:5021]
dataset2_images += six_images[250:500]
dataset2_images += seven_images[250:500]
dataset2_images += eight_images[250:500]
dataset2_images += nine_images[250:500]

dataset2_labels += zero_labels[5000:5250]
dataset2_labels += one_labels[5000:5250]
dataset2_labels += two_labels[5000:5250]
dataset2_labels += three_labels[250:5250]
dataset2_labels += four_labels[250:5250]
dataset2_labels += five_labels[150:5021]
dataset2_labels += six_labels[250:500]
dataset2_labels += seven_labels[250:500]
dataset2_labels += eight_labels[250:500]
dataset2_labels += nine_labels[250:500]

dataset3_images += zero_images[5250:]
dataset3_images += one_images[5250:5950]
dataset3_images += two_images[5250:5950]
dataset3_images += three_images[5250:5950]
dataset3_images += four_images[5250:]
dataset3_images += five_images[5021:]
dataset3_images += six_images[500:4300]
dataset3_images += seven_images[500:4300]
dataset3_images += eight_images[500:4300]
dataset3_images += nine_images[500:4300]

dataset3_labels += zero_labels[5250:]
dataset3_labels += one_labels[5250:5950]
dataset3_labels += two_labels[5250:5950]
dataset3_labels += three_labels[5250:5950]
dataset3_labels += four_labels[5250:]
dataset3_labels += five_labels[5021:]
dataset3_labels += six_labels[500:4300]
dataset3_labels += seven_labels[500:4300]
dataset3_labels += eight_labels[500:4300]
dataset3_labels += nine_labels[500:4300]

combined1 = list(zip(dataset1_images, dataset1_labels))
combined2 = list(zip(dataset2_images, dataset2_labels))
combined3 = list(zip(dataset3_images, dataset3_labels))

random.shuffle(combined1)
random.shuffle(combined2)
random.shuffle(combined3)

dataset1_images[:], dataset1_labels[:] = zip(*combined1)
dataset2_images[:], dataset2_labels[:] = zip(*combined2)
dataset3_images[:], dataset3_labels[:] = zip(*combined3)

[5923, 6742, 5958, 6131, 5842, 5421, 5918, 6265, 5851, 5949]


In [7]:
# Preprocess and group data and label
'''
# Lists to Split Training for Testing
data_list = [np.asarray(train_input_pixels), np.asarray(dataset1_images), np.asarray(dataset2_images),
             np.asarray(dataset3_images)]
label_list = [train_labels, dataset1_labels, dataset2_labels, dataset3_labels]

epoch_list, train_list, label_list = with_train_data(data_list, label_list)
'''

# Lists with Training and Testing Available
data_list = [np.asarray(train_input_pixels), np.asarray(dataset1_images), np.asarray(dataset2_images),
             np.asarray(dataset3_images), np.asarray(test_input_pixels)]
label_list = [train_labels, dataset1_labels, dataset2_labels, dataset3_labels, test_labels]

# Add preprocessed epochs, padded, one hot encoded into lists
# 3rd arg is epoch for whole dataset
epoch_list, train_list, label_list = with_train_and_test_data(data_list, label_list, 10) 

In [ ]:
# Load in trained model
model_list = [] 
model_list.append(load_model(model_dir + 'model_whole-0.01LR.h5', custom_objects={'approxReLU': approxReLU}))
model_list.append(load_model(model_dir + 'model1-0.01LR.h5', custom_objects={'approxReLU': approxReLU}))
model_list.append(load_model(model_dir + 'model2-0.01LR.h5', custom_objects={'approxReLU': approxReLU}))
model_list.append(load_model(model_dir + 'model3-0.01LR.h5', custom_objects={'approxReLU': approxReLU}))

In [ ]:
# Train whole, model1, model2, model3
model_list = train_dataset(epoch_list, train_list, label_list, custom=False)

W0728 20:47:32.749178 24024 deprecation_wrapper.py:119] From C:\Users\PandaBear\Anaconda3\envs\python37\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0728 20:47:32.761177 24024 deprecation_wrapper.py:119] From C:\Users\PandaBear\Anaconda3\envs\python37\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0728 20:47:32.763180 24024 deprecation_wrapper.py:119] From C:\Users\PandaBear\Anaconda3\envs\python37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0728 20:47:32.825179 24024 deprecation_wrapper.py:119] From C:\Users\PandaBear\Anaconda3\envs\python37\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0728 20

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 29, 29, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 5)         130       
_________________________________________________________________
flatten_1 (Flatten)          (None, 845)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               84600     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 85,740
Trainable params: 85,740
Non-trainable params: 0
_________________________________________________________________
Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 11s 220us/step - loss: 0.6357 

Epoch 21/36
13320/13320 [==============================] - 1s 104us/step - loss: 0.0557 - acc: 0.9845 - val_loss: 0.0987 - val_acc: 0.9787
Epoch 22/36
13320/13320 [==============================] - 2s 128us/step - loss: 0.0541 - acc: 0.9848 - val_loss: 0.1020 - val_acc: 0.9757
Epoch 23/36
13320/13320 [==============================] - 2s 122us/step - loss: 0.0522 - acc: 0.9854 - val_loss: 0.0914 - val_acc: 0.9784
Epoch 24/36
13320/13320 [==============================] - 1s 110us/step - loss: 0.0497 - acc: 0.9864 - val_loss: 0.0897 - val_acc: 0.9796
Epoch 25/36
13320/13320 [==============================] - 1s 106us/step - loss: 0.0486 - acc: 0.9860 - val_loss: 0.0887 - val_acc: 0.9763
Epoch 26/36
13320/13320 [==============================] - 1s 105us/step - loss: 0.0461 - acc: 0.9875 - val_loss: 0.0883 - val_acc: 0.9811
Epoch 27/36
13320/13320 [==============================] - 1s 109us/step - loss: 0.0442 - acc: 0.9884 - val_loss: 0.0869 - val_acc: 0.9805
Epoch 28/36
13320/13320 [==

15172/15172 [==============================] - 2s 102us/step - loss: 0.1695 - acc: 0.9509 - val_loss: 0.1701 - val_acc: 0.9504
Epoch 8/31
15172/15172 [==============================] - 2s 102us/step - loss: 0.1575 - acc: 0.9552 - val_loss: 0.1797 - val_acc: 0.9473
Epoch 9/31
15172/15172 [==============================] - 2s 101us/step - loss: 0.1480 - acc: 0.9582 - val_loss: 0.1615 - val_acc: 0.9483
Epoch 10/31
15172/15172 [==============================] - 2s 102us/step - loss: 0.1411 - acc: 0.9596 - val_loss: 0.1467 - val_acc: 0.9578
Epoch 11/31
15172/15172 [==============================] - 2s 102us/step - loss: 0.1327 - acc: 0.9607 - val_loss: 0.1502 - val_acc: 0.9562
Epoch 12/31
15172/15172 [==============================] - 2s 102us/step - loss: 0.1249 - acc: 0.9648 - val_loss: 0.1329 - val_acc: 0.9594
Epoch 13/31
15172/15172 [==============================] - 2s 102us/step - loss: 0.1190 - acc: 0.9659 - val_loss: 0.1402 - val_acc: 0.9565
Epoch 14/31
15172/15172 [================

In [ ]:
# Save trained model
model_list[0].save(model_dir + 'model_whole-0.0095LR.h5')
model_list[1].save(model_dir + 'model1-0.0095LR.h5')
model_list[2].save(model_dir + 'model2-0.0095LR.h5')
model_list[3].save(model_dir + 'model3-0.0095LR.h5')

In [ ]:
# Obtain accuracy parameters to print
test_size, whole_acc, avg_ensemble_acc, max_ensemble_acc, model1_acc, model2_acc, model3_acc = accuracy_test(model_list, train_list[4], label_list[4])

In [ ]:
print("Entire Dataset Acc: " + str(whole_acc) + "/" + str(test_size) +
  '\nAvg Ensemble Acc: ' + str(avg_ensemble_acc) + "/" + str(test_size)+
  '\nMax Ensemble Acc: ' + str(max_ensemble_acc) + "/" + str(test_size)+
  '\nModel 1 Acc: ' + str(model1_acc) + "/" + str(test_size) +
  '\nModel 2 Acc: ' + str(model2_acc) + "/" + str(test_size) +
  '\nModel 3 Acc: ' + str(model3_acc) + "/" + str(test_size))

In [ ]:
model.evaluate(x=test_x1, y=test_y1, verbose=1)
model1.evaluate(x=test_x1, y=test_y1, verbose=1)
model2.evaluate(x=test_x2, y=test_y2, verbose=1)
model3.evaluate(x=test_x3, y=test_y3, verbose=1)

In [ ]:
weights = model1.layers[0].get_weights()[0]
biases = model1.layers[0].get_weights()[1]

weights = model2.layers[0].get_weights()[0]
biases = model2.layers[0].get_weights()[1]

weights = model3.layers[0].get_weights()[0]
biases = model3.layers[0].get_weights()[1]